In [13]:

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Drop the non-beneficial ID columns
application_df.drop(columns=['EIN', 'NAME', 'STATUS'], inplace=True)


In [15]:
# Determine the number of unique values in each column
unique_counts = application_df.nunique()

# Display the number of unique values in each column
print(unique_counts)


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [16]:
# Look at APPLICATION_TYPE value counts
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts
print(application_type_counts)


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [17]:
# Choose a cutoff value for binning
cutoff_value = 500  # Adjust this value as needed

# Get application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace application types in the DataFrame
application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other", inplace=True)

# Check the value counts after binning
print(application_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [18]:
# Look at CLASSIFICATION value counts
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the value counts
print(classification_counts)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [19]:
# Filter CLASSIFICATION value counts to include only values with counts > 1
classification_counts_filtered = classification_counts[classification_counts > 1]

# Display the filtered value counts
print(classification_counts_filtered)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [20]:
# Choose a cutoff value for binning
classification_cutoff_value = 1000  # Adjust this value as needed

# Get classifications to be replaced with "Other"
classifications_to_replace = classification_counts[classification_counts < classification_cutoff_value].index.tolist()

# Replace classifications in the DataFrame
application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other", inplace=True)

# Check the value counts after binning
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [21]:
# Convert categorical data to numeric using pd.get_dummies
application_df_encoded = pd.get_dummies(application_df)



In [22]:
# Define features (X) and target (y)
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = application_df_encoded['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Display the shapes of the training and testing datasets
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)


Training data shape: (25724, 42) (25724,)
Testing data shape: (8575, 42) (8575,)


In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [42]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 128)            │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,873 (62.00 KB)

 Trainable params: 15,873 (62.00 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [56]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, shuffle=True, verbose=2)


Epoch 1/100
804/804 - 2s - 3ms/step - accuracy: 0.7428 - loss: 0.5303
Epoch 2/100
804/804 - 2s - 3ms/step - accuracy: 0.7427 - loss: 0.5309
Epoch 3/100
804/804 - 2s - 3ms/step - accuracy: 0.7426 - loss: 0.5315
Epoch 4/100
804/804 - 3s - 3ms/step - accuracy: 0.7431 - loss: 0.5304
Epoch 5/100
804/804 - 4s - 5ms/step - accuracy: 0.7426 - loss: 0.5302
Epoch 6/100
804/804 - 2s - 3ms/step - accuracy: 0.7428 - loss: 0.5310
Epoch 7/100
804/804 - 2s - 3ms/step - accuracy: 0.7421 - loss: 0.5424
Epoch 8/100
804/804 - 2s - 3ms/step - accuracy: 0.7421 - loss: 0.5315
Epoch 9/100
804/804 - 3s - 4ms/step - accuracy: 0.7429 - loss: 0.5297
Epoch 10/100
804/804 - 3s - 3ms/step - accuracy: 0.7430 - loss: 0.5299
Epoch 11/100
804/804 - 2s - 3ms/step - accuracy: 0.7428 - loss: 0.5299
Epoch 12/100
804/804 - 2s - 3ms/step - accuracy: 0.7433 - loss: 0.5298
Epoch 13/100
804/804 - 4s - 5ms/step - accuracy: 0.7434 - loss: 0.5306
Epoch 14/100
804/804 - 2s - 3ms/step - accuracy: 0.7426 - loss: 0.5299
Epoch 15/100
80

In [57]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Print the model's loss and accuracy
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Print the model's training history
print(fit_model.history)


268/268 - 1s - 2ms/step - accuracy: 0.7258 - loss: 0.6273
Loss: 0.6272934079170227, Accuracy: 0.7258309125900269
{'accuracy': [0.7428471446037292, 0.7426916360855103, 0.7425750494003296, 0.7430803775787354, 0.7425750494003296, 0.7427694201469421, 0.7421085238456726, 0.742069661617279, 0.7429248690605164, 0.7430415153503418, 0.7427694201469421, 0.7432747483253479, 0.7433525323867798, 0.7425750494003296, 0.7430026531219482, 0.7431970238685608, 0.7426916360855103, 0.7431192398071289, 0.7425750494003296, 0.7428082823753357, 0.7428082823753357, 0.7425750494003296, 0.7427304983139038, 0.7428860068321228, 0.7427694201469421, 0.7430415153503418, 0.7425361275672913, 0.742380678653717, 0.7424972653388977, 0.7426527738571167, 0.7434691190719604, 0.7428082823753357, 0.7431970238685608, 0.7426139116287231, 0.7426916360855103, 0.7431581616401672, 0.7429637908935547, 0.7432747483253479, 0.7426916360855103, 0.7431581616401672, 0.7423028945922852, 0.7424972653388977, 0.7429637908935547, 0.7430026531219

In [58]:
# Export our model to HDF5 file
# Define the file path for saving the model
model_file = "AlphabetSoupCharity2.h5"

# Save the model to HDF5 file
nn.save(model_file)

# Print a confirmation message
print(f"Model saved to {model_file}")

Model saved to AlphabetSoupCharity2.h5
